### 数据预处理

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [1]:
#在数据集中的 20 个可用类别中只有 4 个类别
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']

#加载与这些类别匹配的文件列表
from sklearn.datasets import fetch_20newsgroups

In [3]:
# cross valida
# 返回的数据集是一个scikit-learn“ buntch, 其字段可以作为 pythondict 键或object属性访问
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [4]:
# 以便方便，用 target_names保存请求的类别名称列表
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [6]:
len(twenty_train.data)

2257

In [7]:
len(twenty_train.filenames)

2257

In [11]:
# 打印第一个加载文件的第一行
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [14]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


In [15]:
# 监督学习算法将需要训练集的类别标签。
# 类别是新闻组的名称，也恰好是保存单个文档的文件夹的名称。

# 将目标属性加载为与列表中类别名称的索引相对应的整数数组target_names。
# 每个样本的类别整数 id 存储在target属性中：

twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2])

In [16]:
# 也可以取回类别名称
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


## 特征提取

### 将文本内容转化为数字特征向量

- 用bag of words
- 每个单词分配一个固定的整数 id
- 对于每个文档#i，计算每个单词w的出现次数并将其存储为特征值 X[i,j], j是单词w在字典里的索引 即id

- X 中的大多数值将为零，因为对于给定的文档，将使用少于几千个不同的单词
- 所以bag of words通常是 高维稀疏数据集

- scipy.sparse矩阵是完全执行此操作的数据结构

In [17]:
# 用scikit-learn标记文本

# 文本预处理、标记化和停用词过滤都包含在  CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [18]:
# 构建了一个特征索引字典
count_vect.vocabulary_.get(u'algorithm')

4690

### 计数频率

- tc：文档中每个单词的出现次数除以文档中的总单词数
- tf= 1+log10 tc (if tc>0)
- df: 单词出现过的文档数
- idf: logN/df (N 总文档数)
- tf–idf： tf* idf

In [19]:
# 用TfidfTransformer.fit_transform
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_train_tfidf.shape

(2257, 35788)

## 训练分类器

### 朴素贝叶斯

In [20]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [21]:
# 调用transform
docs_new = ['God is love', 'OpenGL on the GPU is fast']

X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


## 构建pipeline

为了使矢量化器 => 转换器 => 分类器更易于使用， scikit-learn提供了一个Pipeline

In [22]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

# vect,tfidf和clf(分类器) 是任意的

In [23]:
# 训练模型

text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

## 评估测试集的性能